In [58]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/goodbooks10k-bookscsv/books.csv


In [59]:
df = pd.read_csv('/kaggle/input/goodbooks10k-bookscsv/books.csv')
df.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [60]:
df.duplicated().sum()

0

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   book_id                    10000 non-null  int64  
 1   goodreads_book_id          10000 non-null  int64  
 2   best_book_id               10000 non-null  int64  
 3   work_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   isbn                       9300 non-null   object 
 6   isbn13                     9415 non-null   float64
 7   authors                    10000 non-null  object 
 8   original_publication_year  9979 non-null   float64
 9   original_title             9415 non-null   object 
 10  title                      10000 non-null  object 
 11  language_code              8916 non-null   object 
 12  average_rating             10000 non-null  float64
 13  ratings_count              10000 non-null  int6

In [62]:
df[df['ratings_count'] == df['ratings_1'] + df['ratings_2'] + df['ratings_3'] + df['ratings_4'] + df['ratings_5']].shape

(14, 23)

In [63]:
df[df['average_rating'] == (df['ratings_1'] + 2*df['ratings_2'] + 3*df['ratings_3'] + 4*df['ratings_4'] + 5*df['ratings_5'])/df['ratings_count']].shape

(0, 23)

In [64]:
df['new_col'] = df['ratings_1'] + df['ratings_2'] + df['ratings_3'] + df['ratings_4'] + df['ratings_5']
df[df['average_rating'] == (df['ratings_1'] + 2*df['ratings_2'] + 3*df['ratings_3'] + 4*df['ratings_4'] + 5*df['ratings_5'])/df['new_col']].shape

(1, 24)

In [65]:
df[df['title'].duplicated()].index.values

array([1291, 2400, 3801, 3911, 4211, 4270, 4374, 4938, 5173, 5267, 5479,
       5616, 5786, 5831, 6480, 6647, 6661, 7002, 7022, 7024, 7448, 7461,
       7644, 7683, 7760, 8167, 8209, 8220, 8743, 8898, 8968, 9111, 9119,
       9296, 9783, 9812])

In [66]:
# prepossing
# takeing columns authors, original_publication_year, original_title, title, image_url
# make a new column rating with ratings_1, ratings_2, ratings_3, ratings_4, ratings_5
# drop duplicated title

# formating
# change name of original_publication_year to year
# change dtype of original_publication_year from float64 to int64
# make a new column tags with original_title, authors
# make a new column similar_books with cosine_similarity

In [67]:
# prepossinng
df = df.iloc[:,[7,8,9,10,16,17,18,19,20,21]]
df['rating'] = round(((df['ratings_1'] + 2*df['ratings_2'] + 3*df['ratings_3'] + 4*df['ratings_4'] + 5*df['ratings_5'])/(df['ratings_1'] + df['ratings_2'] + df['ratings_3'] + df['ratings_4'] + df['ratings_5'])),1)
df.drop(columns=['ratings_1','ratings_2','ratings_3','ratings_4','ratings_5'],inplace=True)
df.drop_duplicates(subset=['title'],inplace=True)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9964 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   authors                    9964 non-null   object 
 1   original_publication_year  9943 non-null   float64
 2   original_title             9381 non-null   object 
 3   title                      9964 non-null   object 
 4   image_url                  9964 non-null   object 
 5   rating                     9964 non-null   float64
dtypes: float64(2), object(4)
memory usage: 544.9+ KB


In [69]:
def collapse(L):
    L1 = ""
    for i in L:
        L1 = L1 + (i.replace(" ",""))
    return L1

In [70]:
# formating
df.rename(columns={'original_publication_year': 'year'},inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df = df.astype({"year": int})
df['tags'] = df['authors'].apply(collapse) + ", " + df['original_title']
df.head()

,authors,year,original_title,title,image_url,rating,tags
0,Suzanne Collins,2008,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",https://images.gr-assets.com/books/1447303603m...,4.3,"SuzanneCollins, The Hunger Games"
1,"J.K. Rowling, Mary GrandPré",1997,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,https://images.gr-assets.com/books/1474154022m...,4.4,"J.K.Rowling,MaryGrandPré, Harry Potter and the..."
2,Stephenie Meyer,2005,Twilight,"Twilight (Twilight, #1)",https://images.gr-assets.com/books/1361039443m...,3.6,"StephenieMeyer, Twilight"
3,Harper Lee,1960,To Kill a Mockingbird,To Kill a Mockingbird,https://images.gr-assets.com/books/1361975680m...,4.3,"HarperLee, To Kill a Mockingbird"
4,F. Scott Fitzgerald,1925,The Great Gatsby,The Great Gatsby,https://images.gr-assets.com/books/1490528560m...,3.9,"F.ScottFitzgerald, The Great Gatsby"


In [71]:
x = df['tags'].values.tolist()
y = ''
for i in x:
    y = y + i
    
len(set(y.split()))

17348

In [72]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vector = cv.fit_transform(df['tags']).toarray()

from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [73]:
def recommend(books):
    L = []
    index = df[df['title'] == books].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key= lambda x: x[1])
    for i in distances[1:11]:
        L.append(df.iloc[i[0]].title)
    
    distances.clear()
    return L

In [75]:
df['similar_books'] = df['title'].apply(recommend)
df.head()

,authors,year,original_title,title,image_url,rating,tags,similar_books
0,Suzanne Collins,2008,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",https://images.gr-assets.com/books/1447303603m...,4.3,"SuzanneCollins, The Hunger Games",[The Hunger Games Trilogy Boxset (The Hunger G...
1,"J.K. Rowling, Mary GrandPré",1997,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,https://images.gr-assets.com/books/1474154022m...,4.4,"J.K.Rowling,MaryGrandPré, Harry Potter and the...",[Harry Potter and the Goblet of Fire (Harry Po...
2,Stephenie Meyer,2005,Twilight,"Twilight (Twilight, #1)",https://images.gr-assets.com/books/1361039443m...,3.6,"StephenieMeyer, Twilight","[The Twilight Collection (Twilight, #1-3), The..."
3,Harper Lee,1960,To Kill a Mockingbird,To Kill a Mockingbird,https://images.gr-assets.com/books/1361975680m...,4.3,"HarperLee, To Kill a Mockingbird","[Mockingbird, First to Kill (Nathan McBride, #..."
4,F. Scott Fitzgerald,1925,The Great Gatsby,The Great Gatsby,https://images.gr-assets.com/books/1490528560m...,3.9,"F.ScottFitzgerald, The Great Gatsby","[Kitchens of the Great Midwest, Great Expectat..."


In [94]:
def fetch_data(book,n):
    l = df[df['title'] == book]['similar_books'].values[0]
    authors = []
    year = []
    image_url = []
    rating = []
    for i in range(n):
        authors.append(df[df['title'] == l[i]].authors.values[0])
        year.append(df[df['title'] == l[i]].year.values[0])
        image_url.append(df[df['title'] == l[i]].image_url.values[0])
        rating.append(df[df['title'] == l[i]].rating.values[0])

    
    return authors,year,image_url,rating

In [95]:
fetch_data('The Great Gatsby',4)

(['J. Ryan Stradal', 'Charles Dickens', 'C.S. Lewis', 'F. Scott Fitzgerald'],
 [2015, 1860, 1945, 1920],
 ['https://images.gr-assets.com/books/1428066565m/23398625.jpg',
  'https://images.gr-assets.com/books/1327920219m/2623.jpg',
  'https://images.gr-assets.com/books/1166805741m/17267.jpg',
  'https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png'],
 [3.8, 3.7, 4.3, 3.7])

In [96]:
books = df.drop(columns=['original_title','tags'])
books.shape

(9375, 6)

In [97]:
import pickle
pickle.dump(books,open('books.pkl','wb'))